In [2]:
from sklearn.utils import shuffle
import numpy as np
import tensorflow as tf

/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
M = 2
K = 3
n = 100
N = n * K

In [4]:
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])
Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

In [8]:
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [10]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), reduction_indices=[1])) 

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [13]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

In [15]:
batch_size = 50
n_batches = N // batch_size

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for epoch in range(20):
    X_, Y_ = shuffle(X, Y)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x:X_[start:end],
            t:Y_[start:end]
        })

In [18]:
X_, Y_ = shuffle(X, Y)

classified = correct_prediction.eval(session=sess, feed_dict={x: X_[0:10], t: Y_[0:10]})
prob = y.eval(session=sess, feed_dict={x: X_[0:10]})
print('classified: \n', classified)
print()
print('output probability: \n', prob)

classified: 
 [ True  True  True  True  True  True  True  True  True  True]

output probability: 
 [[  3.53779089e-10   2.22938973e-03   9.97770667e-01]
 [  7.46819451e-02   9.20228422e-01   5.08960616e-03]
 [  2.34573451e-03   9.25179183e-01   7.24750534e-02]
 [  7.57955573e-11   4.22595302e-04   9.99577463e-01]
 [  5.03602251e-03   9.77156460e-01   1.78074967e-02]
 [  9.98733819e-01   1.26621465e-03   3.05645093e-10]
 [  6.24510236e-02   9.28155363e-01   9.39360354e-03]
 [  1.53760901e-02   9.11862433e-01   7.27615505e-02]
 [  2.32657269e-02   9.44902062e-01   3.18322815e-02]
 [  1.31470085e-10   4.37288109e-04   9.99562681e-01]]


## Keras

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

Using TensorFlow backend.


In [20]:
model = Sequential()
model.add(Dense(input_dim=M, units=K))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.1))

In [21]:
minibatch_size=50
model.fit(X, Y, epochs=20, batch_size=minibatch_size)

Epoch 1/20
300/300 [==============================] - 0s 172us/step - loss: 2.1562
Epoch 2/20
300/300 [==============================] - 0s 33us/step - loss: 0.1822
Epoch 3/20
300/300 [==============================] - 0s 24us/step - loss: 0.1309
Epoch 4/20
300/300 [==============================] - 0s 29us/step - loss: 0.1084
Epoch 5/20
300/300 [==============================] - 0s 21us/step - loss: 0.0902
Epoch 6/20
300/300 [==============================] - 0s 18us/step - loss: 0.0786
Epoch 7/20
300/300 [==============================] - 0s 17us/step - loss: 0.0700
Epoch 8/20
300/300 [==============================] - 0s 18us/step - loss: 0.0639
Epoch 9/20
300/300 [==============================] - 0s 20us/step - loss: 0.0582
Epoch 10/20
300/300 [==============================] - 0s 23us/step - loss: 0.0541
Epoch 11/20
300/300 [==============================] - 0s 22us/step - loss: 0.0506
Epoch 12/20
300/300 [==============================] - 0s 24us/step - loss: 0.0477
Epoch 13/20


In [23]:
X_, Y_ = shuffle(X, Y)
classes = model.predict_classes(X_[0:10], batch_size=minibatch_size)
prob=model.predict_proba(X_[0:10], batch_size=1)
print('classified: \n', np.argmax(model.predict(X_[0:10]), axis=1) == classes)
print()
print('output probability: \n', prob)

10/10 [==============================] - 0s 791us/step
classified: 
 [ True  True  True  True  True  True  True  True  True  True]

output probability: 
 [[  9.58095193e-01   4.19004373e-02   4.38166035e-06]
 [  9.92684960e-01   7.31501821e-03   1.19502657e-08]
 [  3.73565445e-09   2.76344810e-02   9.72365499e-01]
 [  2.63884244e-03   9.58413064e-01   3.89480405e-02]
 [  9.82392907e-01   1.76069941e-02   6.43718892e-08]
 [  9.12567437e-01   8.74322504e-02   3.49182983e-07]
 [  2.18676882e-10   3.08748079e-03   9.96912479e-01]
 [  3.91869247e-02   9.50916529e-01   9.89645254e-03]
 [  1.98372561e-07   2.31321957e-02   9.76867557e-01]
 [  4.11550514e-03   9.75039363e-01   2.08450779e-02]]
